# Installations and Imports 

In [ ]:
!pip install transformers 
!pip install python-docx
# !pip install transformers ==3.1.0 (for awesomealign)
# !pip install stanza 
# !pip install awesome-align
!pip install laserembeddings
!python -m laserembeddings download-models

import sys
import torch
import transformers
import itertools
import docx
import re
from bs4 import BeautifulSoup
import spacy
# import stanza 
import numpy as np
import pandas as pd
import json
import re 
import nltk
from nltk.tokenize import sent_tokenize
from laserembeddings import Laser

laser = Laser()
nltk.download('punkt')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 86 kB 3.1 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 6.6 MB 25.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 4.9 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=1389da493942859c4e03691d78df8824605ed749e612a098226158e33c48591f
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

# Pre-processing

In [ ]:
def chapFromTEI(soup):
  chap = {}

  for note in soup.find_all('note'):
    note.decompose() 
  # for head in soup.find_all('head'):
  #   head.decompose() Why no head?? 
  for i in range(len(soup.find_all('div', attrs={"type": "textpart", "subtype": "chapter"}))):
    for p in soup.find_all('div', attrs={"type": "textpart", "subtype": "chapter"})[i].find_all('p'):
      text = ''
      if f'Chapter {i+1}' in chap:
        chap[f'Chapter {i+1}'] = chap[f'Chapter {i+1}'] + p.get_text().replace('\n','').replace('                     ', ' ').replace('                  ', ' ').replace('     ', ' ').replace('    ', ' ')
      else:
        chap[f'Chapter {i+1}'] = p.get_text().replace('\n','').replace('                     ', ' ').replace('                  ', ' ').replace('     ', ' ').replace('    ', ' ')

  df = pd.DataFrame.from_dict(chap, orient='index')
  return df

## French

In [ ]:
french_xml = open('/content/bodin.livrep.perseus-fre1.xml')
french_soup = BeautifulSoup(french_xml, 'lxml')

In [ ]:
french_df = chapFromTEI(french_soup)

In [ ]:
with open('dev.fr', "w") as frb:
  for line in french_df[0][0].split('.'):
    frb.write(line + '.\n')

## English

In [ ]:
english_xml = open('/content/bodin.livrep.perseus-eng1.xml')
english_soup = BeautifulSoup(english_xml, 'lxml') 

english_df = chapFromTEI(english_soup)

with open('dev.en', "w") as erb:
  for line in english_df[0][0].split('.'):
    erb.write(line + '.\n')

# Sentence-level (Vecalign)

In [ ]:
!git clone https://github.com/thompsonb/vecalign

Cloning into 'vecalign'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 47 (delta 0), reused 1 (delta 0), pack-reused 44
Unpacking objects: 100% (47/47), done.
Checking out files: 100% (40/40), done.


In [ ]:
!/content/vecalign/overlap.py -i /content/dev.fr -o overlaps.fr -n 10
!/content/vecalign/overlap.py -i /content/dev.en -o overlaps.en -n 10

In file included from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1969:0,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /root/.pyxbld/temp.linux-x86_64-3.7/pyrex/dp_core.c:687:
/usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^~~~~~~


In [ ]:
with open('/content/overlaps.fr', 'r') as frOver:
  fr_overlap = frOver.readlines() 

with open('/content/overlaps.en', 'r') as enOver:
  en_overlap = enOver.readlines()

In [ ]:
fr_embeddings = laser.embed_sentences(fr_overlap,lang='fr')
en_embeddings = laser.embed_sentences(en_overlap,lang='en')

with open('overlaps.fr.emb','wb') as fEmb:
  for line in fr_embeddings:
    fEmb.write(bytes(line))

with open('overlaps.en.emb','wb') as eEmb:
  for line in en_embeddings:
    eEmb.write(bytes(line))   

In [ ]:
old_stdout = sys.stdout

## Fre -> Eng

In [ ]:
def createAlignedUnalignedFrEn(log_file):
  aligned_sent = []

  for line in log_file:
    for pair in re.findall('\[..*\]',line):
      tup_pair = []
      for item in pair.split(':'):
        if ',' in item:
          many = []
          for num in item.split(','):
            many.append(int(num.strip().replace('[','').replace(']','')))
          tup_pair.append(tuple(many))
        else:
          if item != '[]':
            tup_pair.append(int(item.replace('[','').replace(']','')))
          else:
            tup_pair.append('None')
      aligned_sent.append(tuple(tup_pair))

  unaligned_sent_x = []
  unaligned_sent_y = []

  for tup in aligned_sent:
    if tup[1] == 'None':
      unaligned_sent_x.append(tup[0])
      #aligned_sent.remove(tup)
    elif tup[0] == 'None':
      unaligned_sent_y.append(tup[1])
      #aligned_sent.remove(tup)
  
  for tup in aligned_sent:
    if ((tup[0] == 'None') or (tup[1] == 'None')):
      aligned_sent.remove(tup)

  aligned_sent = aligned_sent[:8] + aligned_sent[9:] ## NEED BETTER METHOD WILL NOT WORK FOR LATIN

  return aligned_sent, unaligned_sent_x, unaligned_sent_y

In [ ]:
sys.stdout = open('FrEnVecalignLog.txt','wt')

!/content/vecalign/vecalign.py --src /content/dev.fr --tgt /content/dev.en \
--src_embed /content/overlaps.fr /content/overlaps.fr.emb \
--tgt_embed /content/overlaps.en /content/overlaps.en.emb

sys.stdout.close()
sys.stdout = old_stdout

with open('/content/FrEnVecalignLog.txt','r') as fren_val:
  fren_vec_log = fren_val.readlines()

In [ ]:
fren_aligned_sent, fren_unaligned, eng_unaligned = createAlignedUnalignedFrEn(fren_vec_log)

# Word-level (Awesome align Fine tune -- enfr from AA)

In [ ]:
with open('montaigne.src-tgt.txt', 'r') as h:
  fine_tune_final = h.readlines()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

ft_train, ft_test = train_test_split(fine_tune_final, test_size=0.33, random_state=42)
len(ft_train),len(ft_test)

(100, 50)

In [ ]:
train_txt = open('ft_train.src-tgt.txt', 'w')
for line in ft_train:
  train_txt.write(line+'\n')
train_txt.close()

test_txt = open('ft_test.src-tgt.txt', 'w')
for line in ft_test:
  test_txt.write(line + '\n')
test_txt.close()

In [ ]:
!rm -r /content/models*

rm: cannot remove '/content/models*': No such file or directory


In [ ]:
!pip install awesome-align

In [ ]:
TRAIN_FILE='/content/ft_train.src-tgt.txt'
EVAL_FILE='/content/ft_test.src-tgt.txt'
OUTPUT_DIR='/content/models'

!python /usr/local/lib/python3.7/dist-packages/awesome_align/run_train.py \
    --output_dir=$OUTPUT_DIR \
    --model_name_or_path=bert-base-multilingual-cased \
    --extraction 'softmax' \
    --do_train \
    --train_tlm \
    --train_so \
    --train_data_file=$TRAIN_FILE \
    --per_gpu_train_batch_size 2 \
    --gradient_accumulation_steps 4 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --save_steps 4000 \
    --max_steps 20000 \
    --do_eval \
    --eval_data_file=$EVAL_FILE \

03/26/2022 16:18:57 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
03/26/2022 16:18:57 - INFO - awesome_align.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /root/.cache/torch/awesome-align/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0
03/26/2022 16:18:57 - INFO - awesome_align.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 307

In [ ]:
with open("dev.fr", "r") as french_read:
  french_sent = french_read.readlines()

with open("dev.en", "r") as english_read:
  english_sent = english_read.readlines()

In [ ]:
def AwesomeAlign(file, source_text_index):
  INPUT_FILE=file
  OUTPUT_FILE=f'aligned_numbers/number{source_text_index}.src-tgt.txt'
  OUTPUT_WORD_FILE=f'aligned_words/word{source_text_index}.src-tgt.txt'
  MODEL=f'/content/models' 

  !python /usr/local/lib/python3.7/dist-packages/awesome_align/run_align.py \
    --data_file=$INPUT_FILE \
    --output_file=$OUTPUT_FILE \
    --output_word_file=$OUTPUT_WORD_FILE \
    --model_name_or_path=$MODEL \

In [ ]:
!mkdir aligned_numbers
!mkdir aligned_words
# model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')##CHANGE 
french_words = []
english_words = []
for index, i in enumerate(fren_aligned_sent[:6]):
  src = french_sent[i[0]]
  src = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', src)
  src = re.sub('\s{2,}', ' ', src)

  tgt = english_sent[i[1]]
  tgt = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', tgt)
  tgt = re.sub('\s{2,}', ' ', tgt)

  
  with open(f'sent{index}.src-tgt.txt', 'w') as raw_file:
    raw_file.write(src+"   ||| "+tgt)

  AwesomeAlign(f"/content/sent{index}.src-tgt.txt", index)

  align_pairs = []
  align_indices = []
  with open(f'aligned_numbers/number{index}.src-tgt.txt') as number_file:
    align_pairs = number_file.readlines()

  for pair in align_pairs:
    for p in pair.replace('  \n','').split(' '):
      tup = (int(p.split('-')[0]),int(p.split('-')[1].replace('\n', '')))
      align_indices.append(tup)
  align_indices = sorted(align_indices, key=lambda x: x[0])
  
  sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
  for tup in align_indices:
    french_words.append(sent_src[tup[0]])
    english_words.append(sent_tgt[tup[1]])

  # with open(f'/content/sent{index}.src-tgt.txt') as h:
  #   aligned_string = h.read()

  # x_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[0].strip().split()]
  # y_lang = [tokenizer.tokenize(word) for word in aligned_string.split('   ||| ')[1].strip().split()]

  # x_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in x_lang]
  # y_lang = [''.join(i).replace('##','').replace('[UNK]', '') for i in y_lang]

  # for word in x_lang:
  #   french_words.append(word)

  # for word in y_lang:
  #   english_words.append(word)

d = {'french':french_words,'english':english_words}
word_df = pd.DataFrame(data=d)

mkdir: cannot create directory ‘aligned_numbers’: File exists
mkdir: cannot create directory ‘aligned_words’: File exists
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Extracting: 1it [00:00,  2.15it/s]
Loading the dataset...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader runnin

In [ ]:
word_df.to_csv('AAMontaignemodel.csv')

# Word-level (Awesome Align)

In [ ]:
def AwesomeAlign(file, source_text_index):
  INPUT_FILE=file
  OUTPUT_FILE=f'aligned_numbers/number{source_text_index}.src-tgt.txt'
  OUTPUT_WORD_FILE=f'aligned_words/word{source_text_index}.src-tgt.txt'
  MODEL=f'/content/drive/MyDrive/models' 

  !python /usr/local/lib/python3.7/dist-packages/awesome_align/run_align.py \
    --data_file=$INPUT_FILE \
    --output_file=$OUTPUT_FILE \
    --output_word_file=$OUTPUT_WORD_FILE \
    --model_name_or_path=$MODEL \

In [ ]:
with open("dev.fr", "r") as french_read:
  french_sent = french_read.readlines()

with open("dev.en", "r") as english_read:
  english_sent = english_read.readlines()

In [ ]:
align_indices = [(0,0),(1,1),(2,2),(3,3),(4,4),(5,5)]
model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
french_words = []
english_words = []
for i in align_indices:
  src = french_sent[i[0]]
  src = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', src)
  src = re.sub('\s{2,}', ' ', src)

  tgt = english_sent[i[1]]
  tgt = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', tgt)
  tgt = re.sub('\s{2,}', ' ', tgt)

  sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
  token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
  wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
  ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
  sub2word_map_src = []
  for i, word_list in enumerate(token_src):
    sub2word_map_src += [i for x in word_list]
  sub2word_map_tgt = []
  for i, word_list in enumerate(token_tgt):
    sub2word_map_tgt += [i for x in word_list]

  align_layer = 8
  threshold = 1e-3
  model.eval()
  with torch.no_grad():
    out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
    out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

    dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

    softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
    softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

    softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

  align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
  align_words = set()
  for i, j in align_subwords:
    align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

  # printing
  class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

  for i, j in sorted(align_words):
    french_words.append(sent_src[i])
    english_words.append(sent_tgt[j])
    # print(f'{color.BOLD}{color.BLUE}{sent_src[i]}{color.END}==={color.BOLD}{color.RED}{sent_tgt[j]}{color.END}')

d = {'french':french_words,'english':english_words}
word_df = pd.DataFrame(data=d)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
word_df.to_csv('AAbasemodel.csv')

# Word-level (Awesome align - Historical mBERT) https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-historic-multilingual-cased")
model = AutoModelForMaskedLM.from_pretrained("dbmdz/bert-base-historic-multilingual-cased")

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/561 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-historic-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
with open("dev.fr", "r") as french_read:
  french_sent = french_read.readlines()

with open("dev.en", "r") as english_read:
  english_sent = english_read.readlines()

In [ ]:
align_indices = [(0,0),(1,1),(2,2),(3,3),(4,4),(5,5)]

french_words = []
english_words = []
for i in align_indices:
  src = french_sent[i[0]]
  src = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', src)
  src = re.sub('\s{2,}', ' ', src)

  tgt = english_sent[i[1]]
  tgt = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', tgt)
  tgt = re.sub('\s{2,}', ' ', tgt)

  sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
  token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
  wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
  ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
  sub2word_map_src = []
  for i, word_list in enumerate(token_src):
    sub2word_map_src += [i for x in word_list]
  sub2word_map_tgt = []
  for i, word_list in enumerate(token_tgt):
    sub2word_map_tgt += [i for x in word_list]

  align_layer = 8
  threshold = 1e-3
  model.eval()
  with torch.no_grad():
    out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[1][align_layer][0, 1:-1]
    out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[1][align_layer][0, 1:-1]

    dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

    softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
    softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

    softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

  align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
  align_words = set()
  for i, j in align_subwords:
    align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

  # printing
  class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

  for i, j in sorted(align_words):
    french_words.append(sent_src[i])
    english_words.append(sent_tgt[j])
    # print(f'{color.BOLD}{color.BLUE}{sent_src[i]}{color.END}==={color.BOLD}{color.RED}{sent_tgt[j]}{color.END}')

d = {'french':french_words,'english':english_words}
word_df = pd.DataFrame(data=d)

In [ ]:
pd.set_option('display.max_rows', 500)
word_df

,french,english
0,REPUBLIQUE,Commonweale
1,est,is
2,un,a
3,droit,lawfull
4,gouvernement,government
5,de,of
6,plusieurs,many
7,mesnages,families
8,mesnages,soveraigntie
9,",",","


In [ ]:
word_df.to_csv('AAHistoricalmodel.csv')

# Simalign

In [ ]:
!pip install transformers

In [ ]:
!pip install simalign

In [ ]:
french_words = []
english_words = []

from simalign import SentenceAligner
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

for i in fren_aligned_sent[:6]:
  src = french_sent[i[0]]
  src = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', src)
  src = re.sub('\s{2,}', ' ', src)

  tgt = english_sent[i[1]]
  tgt = re.sub(r'([,.:!?"\'()-<>])', r' \1 ', tgt)
  tgt = re.sub('\s{2,}', ' ', tgt)

  sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
  # token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt] could try with BERT tokenization later

  alignments = myaligner.get_word_aligns(sent_src, sent_tgt)
  
  for tup in alignments['itermax']:
    french_words.append(sent_src[tup[0]])
    english_words.append(sent_tgt[tup[1]])
   
  # for matching_method in alignments:
  #     print(matching_method, ":", alignments[matching_method])

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-03-15 23:44:15,262 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [ ]:
d = {'french':french_words,'english':english_words}
word_df = pd.DataFrame(data=d)
word_df.to_csv('SIMbasemodel.csv')

# Word-level (TM2TB)

In [ ]:
!git clone https://github.com/luismond/tm2tb
%cd tm2tb
!pip install tf-estimator-nightly==2.8.0.dev2021122109
!pip install folium==0.2.1
! pip install -r requirements.txt

from tm2tb import TermExtractor
from tm2tb import BitermExtractor
from tm2tb import BitextReader
import pandas as pd
pd.options.display.width = 200

fatal: destination path 'tm2tb' already exists and is not an empty directory.
/content/tm2tb
  Using cached en_core_web_md-3.2.0-py3-none-any.whl
  Using cached es_core_news_md-3.2.0-py3-none-any.whl
  Using cached fr_core_news_md-3.2.0-py3-none-any.whl
  Using cached de_core_news_md-3.2.0-py3-none-any.whl
  Using cached tokenizers-0.11.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.8 MB)
  Using cached transformers-4.17.0-py3-none-any.whl (3.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc2
    Uninstalling tokenizers-0.8.1rc2:
      Successfully uninstalled tokenizers-0.8.1rc2
  Attempting uninstall: transformers
    Found existing installation: transformers 3.1.0
    Uninstalling transformers-3.1.0:
      Successfully uninstalled transformers-3.1.0


Loading sentence transformer model:
distiluse-base-multilingual-cased-v1
Loading spacy models...


In [ ]:
with open("dev.fr", "r") as french_read:
  french_sent = french_read.readlines()

with open("dev.en", "r") as english_read:
  english_sent = english_read.readlines()

In [ ]:
for i in fren_aligned_sent:
  fr_sentence = french_sent[i[0]]
  en_sentence = english_sent[i[1]]

  extractor = BitermExtractor((fr_sentence, en_sentence))
  biterms = extractor.extract_terms()
  source_terms = biterms['src_term'].tolist()
  target_terms = biterms['trg_term'].tolist()
  print(fr_sentence, en_sentence)
  print(source_terms, target_terms)
  print('\n')

 REPUBLIQUE est un droit gouvernement de plusieurs mesnages, & de ce qui leur est commun, avec puissance souveraine.
 A commonweale is a lawfull government of many families, and of that which unto them in common belongeth, with a puissant soveraigntie.

['gouvernement', 'commun'] ['government', 'common']


 Nous mettons cette definition en premier lieu, par ce qu’il faut chercher en toutes choses la fin principale: & puis apres les moyens d’y parvenir.
  this definition omitted by them which have written of a commonweale, wee have placed in the first place: for that in all arts and actions, it behoveth us first to behold the end, and afterward the meanes to attaine therunto.

['definition', 'premier lieu', 'fin', 'premier', 'lieu'] ['definition', 'first place', 'end', 'first', 'place']


 Or la definition n'est autre chose que la fin du sujet qui se presente: & si elle n'est bien fondee, tout ce qui sera basti sur icelle se ruinera bien tost apres.
  for a definition is nothing else th

TypeError: ignored

# Eval

In [ ]:
def evaluateCSV(csv_file):
  eval = pd.read_csv(csv_file)
  tp = eval.groupby('type').size()['tp']
  fp = eval.groupby('type').size()['fp']

  precision = tp/(tp+fp)

  french_word_list = eval['french'].to_list()
  full_text = ''
  for line in french_sent:
    full_text = full_text + line
  unaligned_words = []
  for word in full_text.replace(',','').replace('.','').replace(':','').split():
    if word not in french_word_list:
      unaligned_words.append(word)
  fn = len(unaligned_words)

  recall = tp/(tp+fn)

  f1 = tp/(tp+(1/2*(fp+fn)))

  return f1, recall, precision

## Awesome Align modern

In [ ]:
evaluateCSV('/content/AAmodernmodelWithType - AAmodernmodel.csv')

(0.11759764804703907, 0.06340579710144928, 0.8092485549132948)

## Awesome Align Historical

In [ ]:
evaluateCSV('/content/AAHistoricalmodelType - AAHistoricalmodel.csv')

(0.11046025104602511, 0.05989110707803993, 0.7096774193548387)

## Simalign

In [ ]:
evaluateCSV('/content/SIMbasemodelwithType - SIMbasemodel.csv')

(0.12121212121212122, 0.06589327146171693, 0.7553191489361702)

## AwesomeAlign

In [ ]:
aa_eval = pd.read_csv('/content/AAbasemodelwithtype - AAbasemodel (1).csv')

In [ ]:
trupos = aa_eval.groupby('type').size()['tp']
falpos = aa_eval.groupby('type').size()['fp']

precision = trupos/(trupos+falpos)
precision

0.7586206896551724

In [ ]:
## Get false negatives -- all unaligned French words
french_word_list = aa_eval['french'].to_list()

full_text = ''
for line in french_sent:
  full_text = full_text + line

unaligned_words = []
for word in full_text.replace(',','').replace('.','').replace(':','').split():
  if word not in french_word_list:
    unaligned_words.append(word)

falneg = len(unaligned_words)
falneg

2035

In [ ]:
recall = trupos/(trupos+falneg)
recall

0.06091370558375635

In [ ]:
f1 = trupos/(trupos+(1/2*(falpos+falneg)))
f1

0.11277231952157198

In [ ]:
## Base line -- from AwesomeAlign github
model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

src = 'Deduisons donc par le menu les parties de la definition , que nous avons posee .'
tgt = 'Wherefore let us well examine the end , and every part of the definition by us before propounded .'

# pre-processing
sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
sub2word_map_src = []
for i, word_list in enumerate(token_src):
  sub2word_map_src += [i for x in word_list]
sub2word_map_tgt = []
for i, word_list in enumerate(token_tgt):
  sub2word_map_tgt += [i for x in word_list]

# alignment
align_layer = 8
threshold = 1e-3
model.eval()
with torch.no_grad():
  out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
  out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

  dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

  softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
  softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

  softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
align_words = set()
for i, j in align_subwords:
  align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

# printing
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

for i, j in sorted(align_words):
  print(f'{color.BOLD}{color.BLUE}{sent_src[i]}{color.END}==={color.BOLD}{color.RED}{sent_tgt[j]}{color.END}')

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Deduisons===examine
par===well
les===the
parties===part
de===of
la===the
definition===definition
,===,
que===by
nous===us
avons===before
posee===propounded
.===.


In [ ]:
## Base line -- from AwesomeAlign github
model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

src = 'Tous les êtres humains naissent libres et égaux en dignité et en droits .'
tgt = 'All human beings are born free and equal in dignity and rights .'

# pre-processing
sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
sub2word_map_src = []
for i, word_list in enumerate(token_src):
  sub2word_map_src += [i for x in word_list]
sub2word_map_tgt = []
for i, word_list in enumerate(token_tgt):
  sub2word_map_tgt += [i for x in word_list]

# alignment
align_layer = 8
threshold = 1e-3
model.eval()
with torch.no_grad():
  out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
  out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

  dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

  softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
  softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

  softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
align_words = set()
for i, j in align_subwords:
  align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

# printing
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

for i, j in sorted(align_words):
  print(f'{color.BOLD}{color.BLUE}{sent_src[i]}{color.END}==={color.BOLD}{color.RED}{sent_tgt[j]}{color.END}')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tous===All
êtres===beings
humains===human
naissent===are
naissent===born
libres===free
et===and
égaux===equal
en===in
dignité===dignity
et===and
droits===rights
.===.
